### In this Lab you will add additional fields to the state to define complex behavior without relying on the message list

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load from .env file

api_key = os.getenv("TAVILY_API_KEY")

print("API key loaded:", api_key[:4] + "..." if api_key else "Not found")

API key loaded: tvly...


In [3]:
from langchain.chat_models import init_chat_model # type: ignore

# Follow the steps here to configure your credentials:
# https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html

llm = init_chat_model(
    "anthropic.claude-3-haiku-20240307-v1:0",
    model_provider="bedrock_converse",
)

### 1- Add keys to the state and update the state inside the tool

In [9]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.checkpoint.memory import MemorySaver

from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

graph_builder = StateGraph(State)

@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)

tool = TavilySearch(max_results=2)
tools =[tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

### 2- Prompt the chatbot

In [10]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'tavily_search', 'input': {'query': 'when was LangGraph released', 'search_depth': 'advanced'}, 'id': 'tooluse_qsHzXSQoR6qULvuRu3E0bw'}]
Tool Calls:
  tavily_search (tooluse_qsHzXSQoR6qULvuRu3E0bw)
 Call ID: tooluse_qsHzXSQoR6qULvuRu3E0bw
  Args:
    query: when was LangGraph released
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "when was LangGraph released", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://juancalvoferrandiz.medium.com/discovering-langgraph-paving-the-path-to-reliable-ai-systems-a9cd348c9d57", "title": "Discovering LangGraph: Paving the

We've hit the interrupt in the human_assistance tool again.

The chatbot failed to identify the correct date, so supply it with information:

### 3- Add human assistance

In [11]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'type': 'text', 'text': 'Based on the search results, it appears that LangGraph was released in mid-January 2024. The first result mentions that "LangGraph was released in mid-January 2024" and provides some context around what LangGraph is and how it relates to Langchain. The second result also references the release of LangGraph, saying it launched in January 2023 and had a stable 0.1 release in June.'}, {'type': 'tool_use', 'name': 'human_assistance', 'input': {'name': 'Human', 'birthday': '1980-01-01'}, 'id': 'tooluse_DeC5eXxGQfiGeH5pgxqPtQ'}]
Tool Calls:
  human_assistance (tooluse_DeC5eXxGQfiGeH5pgxqPtQ)
 Call ID: tooluse_DeC5eXxGQfiGeH5pgxqPtQ
  Args:
    name: Human
    birthday: 1980-01-01
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
==================================

In [12]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

### 4- Manually update the state

LangGraph gives a high degree of control over the application state. For instance, at any point (including when interrupted), you can manually override a key using graph.update_state:

In [13]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f039bbd-4ded-617a-8006-800bb895157b'}}

### 5- View the new value

In [14]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}